In [1]:
import pandas as pd

In [2]:
# read parcels_geography input

p_g = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\PBA50 Final Blueprint Large General Input Data\2020_11_10_parcels_geography.csv')

C:\Users\ywang\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# read parcels what fall into California Conservation Easement: 
# https://arcgis.ad.mtc.ca.gov/portal/home/webmap/viewer.html?layers=60d2f1935c3a466ea7113de2a3295292
# this data is created in ArcGIS by "select by location - p10 parcels whose centroid is within conservation easement"

cons_easement = pd.read_csv(r'M:\Data\Urban\BAUS\PBA50\Final_Blueprint\Zoning Modifications\p10_within_conservation_easement.csv') 
# only keep 'PARCEL_ID' and make sure it's integer
cons_easement = cons_easement[['PARCEL_ID']]
cons_easement.PARCEL_ID = cons_easement.PARCEL_ID.apply(lambda x: int(round(x)))
cons_easement.columns = ['PARCEL_ID_cons_easement']
print('{} parcels are within conservation easement'.format(len(cons_easement.PARCEL_ID_cons_easement.unique())))

3201 parcels are within conservation easement


In [4]:
# merge
nodev_comp = p_g.merge(cons_easement, 
                       left_on='PARCEL_ID',
                       right_on='PARCEL_ID_cons_easement',
                       how='left')

# create a field 'cons_easement' to label parcels within conservation easement
nodev_comp['cons_easement'] = 'not cons_easement'
nodev_comp.loc[nodev_comp.PARCEL_ID_cons_easement.notnull(), 'cons_easement'] = 'cons_easement'

# create a field 'compare' to categorize parcels into the following groups:
#  - 'cons_easement but developable': parcels within conservation easement but still 'developable' in urbansim
#  - 'not cons_easement but nodev': parcels outside of conservation easement but still not developable in urbansim
#  - 'other': other parcels

nodev_comp = nodev_comp[['PARCEL_ID', 'nodev', 'cons_easement']]
nodev_comp['compare'] = 'other'
nodev_comp.loc[(nodev_comp.nodev == 0) & (nodev_comp.cons_easement == 'cons_easement'), 
               'compare'] = 'cons_easement but developable'
nodev_comp.loc[(nodev_comp.nodev == 1) & (nodev_comp.cons_easement == 'not cons_easement'),
               'compare'] = 'not cons_easement but nodev'
display(nodev_comp.head())

,PARCEL_ID,nodev,cons_easement,compare
0,229116,0.0,not cons_easement,other
1,244166,0.0,not cons_easement,other
2,202378,0.0,not cons_easement,other
3,2004420,0.0,not cons_easement,other
4,340332,1.0,not cons_easement,not cons_easement but nodev


In [5]:
# statistics of 'compare'
nodev_comp['compare'].value_counts()

other                            1896127
not cons_easement but nodev        60025
cons_easement but developable         56
Name: compare, dtype: int64

In [6]:
# read Urbansim no-project run parcel-level output

p50_np = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\EIR runs\Baseline Large (s25) runs\NP_v5\run188_parcel_data_2050.csv',
                     usecols = ['parcel_id', 'residential_units', 'totemp'])
p50_np.columns = [x+'_50' for x in list(p50_np)]

p15_np = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\EIR runs\Baseline Large (s25) runs\NP_v5\run188_parcel_data_2015.csv',
                     usecols = ['parcel_id', 'residential_units', 'totemp'])
p15_np.columns = [x+'_15' for x in list(p15_np)]

In [7]:
# join to parcels within conversation easement but still developable
cons_dev_sub = nodev_comp.loc[nodev_comp.compare == 'cons_easement but developable']
cons_dev = cons_dev_sub.merge(p50_np,
                              left_on='PARCEL_ID', 
                              right_on='parcel_id_50', 
                              how='left').merge(p15_np,
                                                left_on='PARCEL_ID',
                                                right_on='parcel_id_15',
                                                how='left')

In [8]:
# fill na and calculate 2015-2050 growth

cons_dev.fillna({'residential_units_50':0,
                 'residential_units_15':0,
                 'totemp_50':0,
                 'totemp_15':0}, inplace=True)
cons_dev['residential_units_add'] = cons_dev['residential_units_50'] - cons_dev['residential_units_15']
cons_dev['totemp_add'] = cons_dev['totemp_50'] - cons_dev['totemp_15']

In [9]:
# check these parcels that had residential growth

print(cons_dev[['residential_units_50','residential_units_15']].sum())
cons_dev.loc[cons_dev.residential_units_add>0][['PARCEL_ID','residential_units_50','residential_units_15','residential_units_add']]

residential_units_50    47.0
residential_units_15    20.0
dtype: float64


,PARCEL_ID,residential_units_50,residential_units_15,residential_units_add
27,388858,28.0,1.0,27.0


In [10]:
print(cons_dev[['totemp_50','totemp_15']].sum())
cons_dev.loc[cons_dev.totemp_add>0][['PARCEL_ID','totemp_50','totemp_15','totemp_add']]

totemp_50    21.0
totemp_15    13.0
dtype: float64


,PARCEL_ID,totemp_50,totemp_15,totemp_add
2,1904139,3.0,2.0,1.0
7,358206,3.0,0.0,3.0
21,239675,5.0,0.0,5.0


In [11]:
# export the 'cons_easement but developable' parcels in order to update the 'parcels_geography' table
cons_dev_export = cons_dev[['PARCEL_ID']]
cons_dev_export['nodev'] = 1
print('export {} records for parcels within cons_easement but were labeled as developable'.format(cons_dev_export.shape[0]))
display(cons_dev_export.head())
cons_dev_export.to_csv(r'M:\Data\Urban\BAUS\PBA50\Final_Blueprint\Zoning Modifications\noDev_parcels_conservation_easement.csv', index=False)

export 56 records for parcels within cons_easement but were labeled as developable


C:\Users\ywang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PARCEL_ID,nodev
0,388856,1
1,794431,1
2,1904139,1
3,662723,1
4,855699,1
